In [ ]:
%load_ext autoreload
%autoreload 2

## Notebook pour tester le pipe de fleet-telemetry

### Raw tss

In [ ]:
from core.s3_utils import *
from transform.raw_tss.fleet_telemetry_raw_tss import *
import pandas as pd
import plotly.express as px
from transform.processed_tss.config import *
from core.constants import *
from transform.raw_tss.config import *
# from transform.processed_tss.ProcessedTimeSeries import ProcessedTimeSeries
from transform.raw_results.tesla_fleet_telemetry import get_results as get_results_origin
from transform.processed_results.main import get_processed_results
from datetime import timedelta

import numpy as np

In [ ]:
S3_RAW_TSS_KEY_FORMAT = "raw_ts/{brand}/time_series/raw_tss.parquet"
FLEET_TELEMETRY_RAW_TSS_KEY = S3_RAW_TSS_KEY_FORMAT.format(brand="tesla-fleet-telemetry")

In [ ]:
FLEET_TELEMETRY_RAW_TSS_KEY

In [ ]:
s3 = S3_Bucket()

In [ ]:
access_key = S3_Bucket().get_creds_from_dot_env()['aws_access_key_id']
secret_key = S3_Bucket().get_creds_from_dot_env()['aws_secret_access_key']

In [ ]:
from pyspark.sql import SparkSession
import os
from transform.raw_tss.fleet_telemetry_raw_tss import *
os.environ["PYSPARK_SUBMIT_ARGS"] = (
    "--packages org.apache.hadoop:hadoop-aws:3.3.4 pyspark-shell"

)
spark = SparkSession.builder \
    .appName("Scaleway S3 Read JSON") \
    .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:3.3.4") \
    .config("spark.hadoop.fs.s3a.endpoint", "https://s3.fr-par.scw.cloud") \
    .config("spark.hadoop.fs.s3a.access.key", access_key) \
    .config("spark.hadoop.fs.s3a.secret.key", secret_key) \
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .config("spark.hadoop.fs.s3a.path.style.access", "true") \
    .config("spark.hadoop.fs.s3a.aws.credentials.provider", "org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider") \
    .config("spark.executor.memory", "12g") \
    .config("spark.driver.memory", "12g") \
    .config("spark.driver.maxResultSize", "4g") \
    .getOrCreate()

## Raw TSS

In [ ]:
Ford = RawTss("ford", spark=spark)

In [ ]:
test = Ford.get_raw_tss(Ford.spark, force_update=True)

### Process TTS Spark

In [ ]:
from transform.processed_tss.ProcessedTimeSeriesSpark import *

In [ ]:
from pyspark.sql.types import IntegerType

In [ ]:
res = ProcessedTimeSeries("tesla-fleet-telemetry", spark=spark, force_update=True)

### Raw res

In [ ]:
from transform.raw_results.RawResults import RawResult

In [ ]:
r = RawResult('tesla-fleet-telemetry', spark=spark)

In [ ]:
from transform.raw_tss.RawTss import *

In [ ]:
spark.read.parquet("s3a://bib-platform-prod-data/raw_ts/tesla-fleet-telemetry/time_series/spark_raw_tss.parquet")